Using the drug-review dataset found at https://www.kaggle.com/datasets/mohamedabdelwahabali/drugreview?resource=download

In [120]:
#pip install numpy pandas nltk cupy scikit-learn spacy
#python -m spacy download en_core_web_sm

In [129]:
import numpy as np
import pandas as pd


train_raw = pd.read_csv("archive/drug_review_train.csv", usecols=["review", "rating"]).to_numpy()
test_raw = pd.read_csv("archive/drug_review_test.csv", usecols=["review", "rating"]).to_numpy()
val_raw = pd.read_csv("archive/drug_review_validation.csv", usecols=["review", "rating"]).to_numpy()

train_raw = train_raw[:22000]       #using 20% of dataset, maintaing rough ratios 
test_raw = test_raw[:9200]  
val_raw = val_raw[:5400]  

print(train_raw.shape)
print(test_raw.shape)
print(val_raw.shape)

(22000, 2)
(9200, 2)
(5400, 2)


In [130]:
def seperate_ratings_and_text(data):
    text = data[:,0]
    ratings = data[:,1]
    return text,ratings


train_raw_text,train_raw_ratings = seperate_ratings_and_text(train_raw)
test_raw_text,test_raw_ratings = seperate_ratings_and_text(test_raw)
val_raw_text,val_raw_ratings = seperate_ratings_and_text(val_raw)


def convert_rating_to_sentiment(rating_list):

    for i in range(len(rating_list)):
        rating = int(rating_list[i] )
        if rating >= 7:
            rating_list[i] = 1
        elif rating <= 4:
            rating_list[i] = -1
        else:
            rating_list[i] = 0

    return rating_list

convert_rating_to_sentiment(train_raw_ratings)
convert_rating_to_sentiment(test_raw_ratings)
convert_rating_to_sentiment(val_raw_ratings)

print(train_raw_text[0])
print(train_raw_ratings[0:100])

"i have used restasis for about a year now and have seen almost no progress.  for most of my life i've had red and bothersome eyes. after trying various eye drops, my doctor recommended restasis.  he said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  when i put the drops in it burns my eyes for the first 30 - 40 minutes.  i've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. every year around spring time my eyes get terrible irritated  and this year has been the same (maybe even worse than other years) even though i've been using restasis for a year now. the only difference i notice was for the first couple weeks, but now i'm ready to move on."
[-1 1 -1 1 1 -1 1 1 1 -1 0 -1 -1 -1 1 1 -1 -1 1 1 1 1 1 1 1 -1 1 1 1 -1 -1
 -1 1 0 0 1 1 1 0 1 -1 1 -1 0 1 1 1 1 1 1 -1 -1 1 -1 1 -1 1 1 0 1 -1 -1 -1
 1 1 1 1 1 1 1 1 1 -1 -1 1 1 1 1 1 1 -1 1 -1 1 -1 1 1 0 1 1 1 -1 1 1 1 1
 -1 1 1 0]


In [131]:
import re

def clean_review(review):
    review = re.sub(r"[^a-zA-Z0-9\s]", "", review)
    review = re.sub(r"\s+", " ", review) 
    review = review.lower()
    return review

def clean_text(text_list):
    cleaned_sentences = [clean_review(sentence) for sentence in text_list]
    return np.array(cleaned_sentences)



train_cleaned = clean_text(train_raw_text)
test_cleaned = clean_text(test_raw_text)
val_cleaned = clean_text(val_raw_text)

print(train_cleaned[0])

i have used restasis for about a year now and have seen almost no progress for most of my life ive had red and bothersome eyes after trying various eye drops my doctor recommended restasis he said it typically takes 3 to 6 months for it to really kick in but it never did kick in when i put the drops in it burns my eyes for the first 30 40 minutes ive talked with my doctor about this and he said it is normal but should go away after some time but it hasnt every year around spring time my eyes get terrible irritated and this year has been the same maybe even worse than other years even though ive been using restasis for a year now the only difference i notice was for the first couple weeks but now im ready to move on


In [132]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

nlp = spacy.load("en_core_web_sm")
lemmatizer = WordNetLemmatizer()
stopWords = set(stopwords.words("english"))

#PoS mapping
pos_mapping = {
    "NOUN": "n",
    "PROPN": "n",
    "VERB": "v",
    "AUX": "v",
    "ADJ": "a",
    "ADV": "r",
}

#cache dictionary for lemmatized words with PoS
lemmatized_cache = {}

def lemmatize_word(word, pos):

    key = (word, pos)  #use word and PoS as the key
    if key in lemmatized_cache:
        return lemmatized_cache[key]

    #compute lemmatized form and cache it
    lemmatized_word = lemmatizer.lemmatize(word, pos_mapping.get(pos, "n"))
    lemmatized_cache[key] = lemmatized_word
    return lemmatized_word

def better_lemmatizer(single_sentence):

    #clean the sentence (assuming clean_review is defined)
    single_sentence = clean_review(single_sentence)

    #get PoS tags for the sentence
    text_PoS = nlp(single_sentence)

    #remove stopwords, lemmatize
    lemmatized_list = [
        lemmatize_word(token.text.lower(), token.pos_)
        for token in text_PoS
        if token.text.lower() not in stopWords
    ]

    return lemmatized_list

def lemmatize(data):

    res = []
    for i in range(len(data)):
        res.append(better_lemmatizer(data[i]))

    return res


# Example usage
train_lem = lemmatize(train_cleaned)  
test_lem = lemmatize(test_cleaned)   
val_lem = lemmatize(val_cleaned)    




In [133]:
train_lem_str = [" ".join(row) for row in train_lem]
test_lem_str = [" ".join(row) for row in test_lem]
val_lem_str = [" ".join(row) for row in val_lem]

print(train_lem_str[0:2])
print(len(train_lem_str))
print(test_lem_str[0:2])
print(len(test_lem_str))

['use restasis year see almost progress life red bothersome eye try various eye drop doctor recommend restasis say typically take 3 6 month really kick never kick put drop burn eye first 30 40 minute talk doctor say normal go away time nt every year around spring time eye get terrible irritated year maybe even bad year even though use restasis year difference notice first couple week ready move', 'experience somewhat mixed use implanon nearly 14 month decide get remove bleed every day day would occasionally stain underwear sheet nt start way first month nt bleed epic two week period everything irregular basically new norm sadly decide get rid implanon endless bleeding mention bleeding usually pretty light bit spot quite period either endless bleeding pretty side effect free except minor acne nt get pregnant yeah butblood lot blood']
22000
['try antidepressant year citalopram fluoxetine amitriptyline none help depression insomnia anxiety doctor suggest change onto 45 mg mirtazapine medi

In [152]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,3),max_features=60000)

train_count = vectorizer.fit_transform(train_lem_str)
test_count = vectorizer.transform(test_lem_str)

val_count = vectorizer.transform(val_lem_str)

In [153]:
print(vectorizer.get_feature_names_out())
print(train_count.shape)
print(test_count.shape)
print(train_count[0])

['0025' '005' '01' ... 'zyrtec' 'zyrtec claritin' 'zyvox']
(22000, 60000)
(9200, 60000)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 86 stored elements and shape (1, 60000)>
  Coords	Values
  (0, 55007)	0.0782652366046615
  (0, 43713)	0.3905098816753789
  (0, 59138)	0.15826783413160608
  (0, 44820)	0.05196490004945877
  (0, 2982)	0.05091969062790188
  (0, 41892)	0.09819157006651072
  (0, 28489)	0.0448272041263182
  (0, 43209)	0.0788331825125084
  (0, 7411)	0.11265263742439997
  (0, 17578)	0.2966802735455527
  (0, 53990)	0.03967630388119018
  (0, 55554)	0.09738205231558633
  (0, 14418)	0.14882005901887777
  (0, 13627)	0.08126389439744283
  (0, 43072)	0.050701717002484235
  (0, 44364)	0.0913551774505362
  (0, 54654)	0.10884915093107465
  (0, 50356)	0.023911480007773038
  (0, 33054)	0.03187234107363963
  (0, 42719)	0.045188683533364374
  (0, 27155)	0.1585111290871082
  (0, 35011)	0.04548217105830427
  (0, 42121)	0.05014925456112923
  (0, 7816)	0.07093645010943733
  (0, 19

In [154]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

model = MultinomialNB()
model.fit(train_count, train_raw_ratings.astype(int))

test_predictions = model.predict(test_count)

print("Test Set Evaluation:")
print("Accuracy:", accuracy_score(test_raw_ratings.astype(int), test_predictions))
print("\nClassification Report:")
print(classification_report(test_raw_ratings.astype(int), test_predictions))

Test Set Evaluation:
Accuracy: 0.673695652173913

Classification Report:
              precision    recall  f1-score   support

          -1       0.96      0.06      0.12      2325
           0       0.00      0.00      0.00       823
           1       0.67      1.00      0.80      6052

    accuracy                           0.67      9200
   macro avg       0.54      0.35      0.31      9200
weighted avg       0.68      0.67      0.56      9200



c:\Users\vihaa\Documents\GitHub (not onedrive)\Sentiment-Analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vihaa\Documents\GitHub (not onedrive)\Sentiment-Analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vihaa\Documents\GitHub (not onedrive)\Sentiment-Analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete